In [1]:
import os, argparse, pickle, librosa
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
import torch
from glob import glob
from eval_utils import *
import models
import IPython.display as ipd
from tqdm import tqdm
from params import get_arg

In [2]:
config = get_arg(['--future', '--diff', '--split_number', '100'])

SR = 8192
WINDOW_SIZE = 500 # us
data_length = config.len
BATCH_SIZE = config.batch
K, m = 8, 8
ls = 128

In [3]:
ABSpath = '/home/skuser/'
path = os.path.join(ABSpath, 'ai_model/pytorch/test_model')
data_path = os.path.join(ABSpath,'datasets/hyundai')
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
accel_raw_data = pickle.load(open(os.path.join(data_path,'stationary_accel_data.pickle'),'rb'))
sound_raw_data = pickle.load(open(os.path.join(data_path,'stationary_sound_data.pickle'),'rb'))
transfer_f = np.array(pickle.load(open(os.path.join(data_path,'transfer_f.pickle'),'rb')))
transfer_f = torch.from_numpy(transfer_f).to(device)
transfer_f.requires_grad = False

In [4]:
dataset = testDataset(accel_raw_data, sound_raw_data, config)
name = f'CombineAutoencoder_sj_S_b{config.b}_d{config.len}_lat{config.latency}_adam_0.001_decay0.7071_featurewav'
if config.future:
    name += '_future'
if config.diff:
    name += f'_diff_weight{config.loss_weight}'
print(name)
modelsavepath = sorted(glob(os.path.join(path, 'model_save/'+name + '/*')), key=lambda x: float(os.path.basename(x).split('.pt')[0].split('_')[-1]))[0]
if config.feature == 'wav':
    model = getattr(models, config.model)(dataset[0][0].shape[1:], dataset[0][1].shape[1:], dataset[0][0].shape[0], dataset[0][1].shape[0], config).to(device)
elif config.feature == 'mel':
    model = getattr(models, config.model)((config.nmels, 12), (config.len,), (config.len + config.b) // (config.nfft // 2) + 1, 8, config).to(device)

CombineAutoencoder_sj_S_b200_d200_lat5_adam_0.001_decay0.7071_featurewav_future_diff_weight0.5


In [5]:
print(modelsavepath)
model.load_state_dict(torch.load(modelsavepath)['model'])
dataset_generator = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=0)

/home/skuser/ai_model/pytorch/test_model/model_save/CombineAutoencoder_sj_S_b200_d200_lat5_adam_0.001_decay0.7071_featurewav_future_diff_weight0.5/29_0.9017.pt


In [6]:
sound = dataset.sound

In [ ]:

data_res, sound_res = [], []
model.eval()
with torch.no_grad():
    for data, sound in tqdm(dataset_generator):
#         data_res.append(conv_with_S(model(data.to(device)), S_data=transfer_f, device=device, config=config).cpu()[:,0,:])
        res = conv_with_S(model(data.to(device)), S_data=transfer_f, device=device, config=config).cpu()

        data_res.append(res[:,config.split_number - (config.split_number // 2):config.split_number + (config.split_number // 2)])
        
#         data_res.append(res[:, config.len // 2:])
        sound_res.append(sound)


100%|█████████▉| 6569/6577 [01:28<00:00, 73.01it/s] 

> /home/skuser/ai_model/pytorch/test_model/eval_utils.py(61)__getitem__()
     59         if sound.size(0) != self.split_num:
     60             pdb.set_trace()
---> 61             sound = torch.cat([sound,torch.zeros((self.split_num - sound.size(0), sound.size(1)), device=sound.device, dtype=sound.dtype)])
     62         return accel.transpose(0,1), sound
     63 



100%|█████████▉| 6569/6577 [01:40<00:00, 73.01it/s]

ipdb> sound.shape
torch.Size([76, 8])
ipdb> idx
210432
ipdb> index
tensor(21043200)
ipdb> self.sound.shape
torch.Size([21043681, 8])
ipdb> sound_index + (self.split_num // 2)
tensor(21043705)
ipdb> sound_index
tensor(21043655)


In [ ]:
from time import time
with torch.no_grad():
    for data, sound in dataset_generator:
        st= time()
        model(data[:1].to(device))
        print(time() - st)
        break

In [ ]:
start_time = config.b+config.len+config.latency+(config.len // 2)-config.split_number
len(sound) - start_time, len(sound_res)

In [ ]:
data_res = torch.cat(data_res)
sound_res = torch.cat(sound_res)

In [ ]:
data_res = torch.reshape(data_res, (-1,data_res.size(-1)))
sound_res = torch.reshape(sound_res, (-1,data_res.size(-1)))

In [ ]:
# 새로 학습시킨 건 이 코드 제외
data_res = data_res[200:]
sound_res = sound_res[:-200]

In [ ]:
fig = plt.figure()
fig, ax_lst = plt.subplots(1, 2, figsize=(16,10))
ax_lst[0].plot(np.arange(data_res.size(0)), data_res[:,0], color="blue")
ax_lst[1].plot(np.arange(sound_res.size(0)), sound_res[:,0], color="red")
# ax_lst[1].plot(np.arange(data_res.size(0)), data_res[:,1])
# ax_lst[0].plot(np.arange(sound_res.size(0)), data_res[:,1])

In [ ]:
start = 1000000
end = 1001000
fig=plt.figure(figsize=(16,8))
ax = fig.add_subplot()
ax.plot(sound_res[start:end,0], color="blue")
ax.plot(data_res[start:end,0], color="red")
ax.plot(sound_res[start:end,0] - data_res[start:end,0], color='green')
# self.writer.add_figure("Train/Plot", fig, data_res.size(0))
plt.show(fig)

In [ ]:
start = 1000000
end = 1010000
fig=plt.figure(figsize=(16,8))
ax = fig.add_subplot()
ax.plot(sound_res[start:end,0] - data_res[start:end,0], color="blue")
# self.writer.add_figure("Train/Plot", fig, data_res.size(0))
plt.show(fig)

In [ ]:
# smoo = ema(data_res, n=40)

In [ ]:
# fig=plt.figure(figsize=(16,8))
# ax = fig.add_subplot()
# ax.plot(smoo[start:end,1], color="red")
# ax.plot(sound_res[start:end,1], color="blue")
# plt.show(fig)

In [ ]:
dBA_metric(sound_res, data_res, True)

In [ ]:
def write_wav(data, sr=8192, name='test_gen'):
    data = data.type(torch.float32).numpy()
    data = data - np.min(data)
    data /= np.max(data)
    write(name+'.wav', sr, data)
    return data


In [ ]:
ipd.Audio('smooth.wav',normalize=False) # load a local WAV file